In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import random

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize, CenterCrop
from tqdm.notebook import tqdm

from pytz import timezone
import datetime as dt

import timm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from f1_score import f1_loss

# import wandb
# wandb.init(project="stratified_imgcls", entity='dayday')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
SEED = 25
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

In [4]:
### Configurations
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/train.csv'
load_path = "/opt/ml/code/saved/epoch15_batch20_scheduler_resnet101e_pretrained2.pt"
mean, std = (0.5, 0.5, 0.5), (0.2, 0.2, 0.2)
log = []
c = ''
time = (dt.datetime.now().astimezone(timezone("Asia/Seoul")).strftime("%Y-%m-%d_%H:%M:%S"))

In [5]:
from albumentations import *
from albumentations.pytorch import ToTensorV2


def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    """
    train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.
    
    Args:
        need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
        img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
        mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
        std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

    Returns:
        transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            CenterCrop(img_size[1],img_size[1]),
            Resize(224,224),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            GaussNoise(p=0.5),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            CenterCrop(img_size[1],img_size[1]),
            Resize(224,224),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [6]:
class teamDataset(Dataset):
    num_classes = 3 * 2 * 3

    def __init__(self, data_path, train = 'train',mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
        self.main_path = data_path
        self.mean = mean
        self.std = std

        df_origin = pd.read_csv(os.path.join(self.main_path, 'train.csv'))
        df_origin['gender'] = df_origin['gender'].map({'male':0, 'female':1})
        df_origin['gender_age_cls'] = df_origin.apply(lambda x : set_age(x['age']) + 3*set_gender(x['gender']) ,axis=1)
        
        train_df, val_df = train_test_split(df_origin, test_size=150, stratify=df_origin.gender_age_cls, random_state=52)

        if train == 'train':
            self.df_csv = train_df
        elif train == 'eval':
            self.df_csv = val_df
        else:
            raise Exception(f'train error {train} not in [''ALL'', ''train'', ''test'']')
    
    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform

    def __getitem__(self, index):
        main_index, sub_index = index//7, index%7
        sub_path = self.df_csv.iloc[main_index]['path']
        file_path = os.path.join(self.main_path, 'images', sub_path)
        files = [file_name for file_name in os.listdir(file_path) if file_name[0] != '.']

        image = Image.open(os.path.join(file_path, files[sub_index]))

        if self.transform:
            image = self.transform(image=np.array(image))['image']

        y = 0

        if (age := self.df_csv.iloc[main_index]['age']) < 30: pass
        elif age >= 30 and age < 60: y += 1
        else : y += 2

        y += self.df_csv.iloc[main_index]['gender'] * 3

        if (mask := files[sub_index][0]) == 'm' : pass
        elif mask == 'i' : y += 6
        elif mask == 'n' : y += 12
        else : raise Exception(f'파일명 오류 : {file_path}, {files[sub_index]}, {mask}')

        return image, y#, age #age는 사진 수동 검증용, 나이 파악이 힘듬

    def __len__(self):
        return len(self.df_csv)*7


class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform
        
    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform
        
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image=np.array(image))['image']
        return image

    def __len__(self):
        return len(self.img_paths)

In [7]:
num_epochs = 1
batch_size = 32
learning_rate = 1e-3
patience = 4
accumulation_steps = 2
best_val_loss = np.inf

test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

test_dataset = TestDataset(image_paths, transform)
test_dataset.set_transform(transform['val'])
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)


In [8]:
# log.append(f'Model         : {res_model.__class__.__name__}')
log.append(f'  load_state  : {load_path}')
log.append(f'Dataset       : {dataset.__class__.__name__}')
# log.append(f'Train_trans   : {TrainTrans.__name__}')
# log.append(f'Test_trans    : {TestTrans.__name__}')
log.append(f'Start_time    : {time}')
log.append(f'Device        : {device}')
# log.append(f'CLASS_NUM     : {CLASS_NUM}')
# log.append(f'NUM_WORKERS   : {NUM_WORKERS}')
log.append(f'BATCH_SIZE    : {batch_size}')
log.append(f'NUM_EPOCH     : {num_epochs}')
# log.append(f'SAVE_INTERVAL : {SAVE_INTERVAL}')


for line in log:
    print(line)

  load_state  : /opt/ml/code/saved/epoch15_batch20_scheduler_resnet101e_pretrained2.pt
Dataset       : MaskBaseDataset
Start_time    : 2021-08-29_19:58:30
Device        : cuda
BATCH_SIZE    : 32
NUM_EPOCH     : 1


In [9]:
# 함수 정의
def set_gender(gender):
    return 0 if gender == 'male' else 1

def set_age(age):
    if age<30:
        return 0
    elif age<60:
        return 1
    else:
        return 2

## 실제 학습 코드

In [ ]:
# wandb init
config={"epochs": num_epochs, "batch_size": batch_size, "learning_rate" : learning_rate}

# team_eval_dataset
team_eval_dataset = teamDataset(
    data_path=data_dir,
    train='eval'
)
team_eval_dataset.set_transform(transform['val'])
team_eval_loader = DataLoader(
    team_eval_dataset,
    batch_size=batch_size,
    shuffle=False
)

# dataset
transform = get_transforms(mean=mean, std=std)
train_dataset = teamDataset(
    data_path=data_dir,
    train='train'
)
train_dataset.set_transform(transform['val'])

# test_dataset
test_dataset = TestDataset(image_paths, transform)
test_dataset.set_transform(transform['val'])
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)


team_eval_preds = [0 for _ in range(len(team_eval_dataset))]
test_preds = [0 for _ in range(len(test_dataset))]
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=SEED)
for fold, (train_ids, valid_ids) in enumerate(skf.split(train_dataset.df_csv, train_dataset.df_csv.gender_age_cls)):
    # Print
    print('--------------------------------')
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # -- Image index
    train_image_ids = sum([[x*7+i for i in range(7)] for x in train_ids],[])
    valid_image_ids = sum([[x*7+i for i in range(7)] for x in valid_ids],[])

    # -- Dataset
    train_dataset = Subset(dataset,train_image_ids)
    valid_dataset = Subset(dataset,valid_image_ids)
    
    # -- DataLoader
    train_loader = data.DataLoader(
        train_dataset,
        batch_size=12,
        shuffle=True
    )
    valid_loader = data.DataLoader(
        valid_dataset,
        batch_size=12,
        shuffle=True
    )

    # -- model
    model = timm.create_model(model_name = "resnest101e", num_classes = 18, pretrained = True)
    # model.load_state_dict(torch.load(load_path))
    model.to(device)

    # -- Loss, optim
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # scheduler = ReduceLROnPlateau(optimizer, 'max',factor = 0.5, patience=2)

    counter = 0
    best_val_acc = 0
    for epoch in range(1):#num_epochs):
        print('*** Epoch {} ***'.format(epoch))
        # Training
        model.train()  
        running_loss, running_acc, running_f1 = 0.0, 0.0, 0.0
            
        for idx, (inputs, labels) in enumerate(tqdm(train_loader)):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # forward
            with torch.set_grad_enabled(True):
                logits = model(inputs)
                _, preds = torch.max(logits, 1)
                loss = criterion(logits, labels)

                loss.backward()
                optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.shape[0]
                running_acc += torch.sum(preds == labels.data)
                running_f1 += f1_score(labels.data.cpu().numpy(), preds.cpu().numpy(), average = 'macro')
                # f1_loss(labels.data, preds)

        epoch_acc = running_acc/len(train_loader.dataset)
        epoch_loss = running_loss/len(train_loader.dataset)
        epoch_f1 = running_f1/len(train_loader)
        print('{} Loss: {:.4f} Acc: {:.4f} F1-score: {:.4f}'.format('train', epoch_loss, epoch_acc, epoch_f1))
        # log.append(f"Train : [FOLD {fold}] Epoch {epoch:0>3d} // (avg) Loss : {epoch_loss:.3f}, Accuracy : {epoch_acc:.3f}, F1 : {epoch_f1:.3f}")
           
        # Validation
        model.eval()
        valid_acc, valid_f1,valid_loss = 0.0, 0.0, 0.0
        for idx, (inputs, labels) in enumerate(tqdm(valid_loader)):
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.set_grad_enabled(False):
                logits = model(inputs)
                _, preds = torch.max(logits, 1)

                # statistics
                valid_acc += torch.sum(preds == labels.data)
                valid_loss += criterion(logits, labels).item()
                valid_f1 += f1_score(labels.data.cpu().numpy(), preds.cpu().numpy(), average = 'macro')
                

        valid_acc /= len(valid_loader.dataset)
        valid_f1 /= len(valid_loader)
        valid_loss /= len(valid_loader.dataset)

        # scheduler.step(valid_acc)

        best_val_loss = min(best_val_loss,valid_loss)
        if valid_acc > best_val_acc:
            print("New best model for val accuracy!")
            print(f"val_acc : {valid_acc}")
            # torch.save(model.state_dict(), f"saved/{fold}fold_{epoch:02}_accuracy_{valid_acc:4.2%}.ckpt")
            best_val_acc = valid_acc
            counter = 0
            # log.append(f"New best model for val accuracy! val_acc : {valid_acc}")
        else:
            counter += 1
        # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
        if counter > patience:
            print("Early Stopping...")
            # log.append('Early Stopping')
            # log.append(c)
            break


        print('{} Acc: {:.4f} f1-score: {:.4f}\n'.format('valid', valid_acc, valid_f1))
        # log.append(f"Valid : [FOLD {fold}] Epoch {epoch:0>3d} // (avg) Loss : {valid_loss:.3f}, Accuracy : {valid_acc:.3f}, F1 : {valid_f1:.3f}")
        # log.append(c)
        # wandb.log({'train_acc': epoch_acc ,'val_acc': val_acc})# 'train_f1': epoch_f1, 'val_f1':val_f1})

    # team_eval_pred
    all_predictions = []
    answers = []
    for images, labels in tqdm(team_eval_loader):
        with torch.no_grad():
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            all_predictions.extend(outputs.cpu().numpy())
            answers.extend(labels.cpu().numpy())
    team_eval_preds = [x+y for x,y in zip(team_eval_preds,all_predictions)]

    # test_pred
    all_predictions = []
    for images in tqdm(test_loader):
        with torch.no_grad():
            images = images.to(device)
            outputs = model(images)
            all_predictions.extend(outputs.cpu().numpy())

    test_preds = [x+y for x,y in zip(test_preds,all_predictions)]

# Check Result
print(f'Team eval accuracy : {torch.sum(torch.tensor(answers) == torch.tensor(team_eval_preds))/len(answers)}, f1-score : {f1_score(answers,team_eval_preds,average="macro")}')
submission['ans'] = np.argmax(test_preds,axis = 1)
# submission.to_csv(os.path.join(test_dir, 'stratified.csv'), index=False)
print('Done')